# importar as bibliotecas

In [1]:
import spacy
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# executar as funções

In [2]:
def searchSimilar(phrase, Comments) -> pandas.DataFrame:
	filterdData = []	
	nlp = spacy.load("pt_core_news_sm")
	corpus_old = (list(Comments['content'].values))
	corpus_old.insert(0, phrase) #concatena a phrase na base dados e coloca a phrase no inicio
    
	similarities = []
    
	for i in corpus_old:
		filterdData.append(clearPhrase(i, nlp, ['PROPN', 'ADJ', 'NOUN', 'VERB'])) #fazendo uma limpeza nos comentários, deixando apenas Pronomes pessoais, adjetivos, substantivos e verbos 

	processedData = TfidfVectorizer().fit_transform(filterdData) #transformo a base de dados em vetores de números TF-IDF
	print(corpus_old[0])
	[lenght, M] = processedData.shape #pego o tamanho da base de dados    
	similaritiesList = []    
	for i in range(1, lenght):
		s = cosine_similarity(processedData[0],processedData[i]) #o comentário processedData[0] é a frase digitada pelo usuário, daí preciso computar a distancia dessa frase com todos os comentários da base de dados
		similarities.append(s)
		if(s != 0): #se a similaridade é zero, nem adiciono o comentário na lista, pois significa que ele é 0 similar
			similaritiesList.append([s, corpus_old[i]])
	Comments['similaridade'] = similarities
    
	return Comments
    
	#return similaritiesList   #retorna uma lista com o comentário e a distancia dele da frase que o usuário digitou
	
def clearPhrase(doc, nlp, pos_tag):
	if(type(doc) != spacy.tokens.doc.Doc):
		doc = nlp(doc)
	return ' '.join([str(t.lemma_) for t in doc if t.pos_ in pos_tag]) 

def addHours(minDate:str,maxDate:str)-> (str,str):
	minDate = minDate + "00:00:00"
	maxDate = maxDate + "23:59:59"

	return minDate,maxDate
def getComments(originPath:str,minDate:str = None,maxDate:str = None,versions = None) -> pandas.DataFrame:
	
	commentsFile = pandas.read_csv(originPath)

	if minDate != None and maxDate != None:
		minDate,maxDate = addHours(minDate,maxDate)

	for i in range(0, len(commentsFile)):
		if commentsFile['date'][i] < minDate or commentsFile['date'][i] > maxDate or commentsFile['version'][i] not in versions:
			#print("droped",commentsFile['version'][i])
			commentsFile = commentsFile.drop([i])
	return commentsFile

def sortbyThumbs(file:pandas.DataFrame) -> pandas.DataFrame:
	file = file.sort_values(by=['thumbs'])
	return file

def limparComentarios(df):
    nlp = spacy.load("pt_core_news_sm")
    lista = []
    for i in df['content']:
        r = clearPhrase(i, nlp, ['PROPN', 'NOUN'])
        lista.append(r) 
    df['comentarios_limpos'] = lista    
    return df

def extrairNgrams(corpus, AA, BB):
    c_vec = CountVectorizer(ngram_range=(AA,BB))
    ngrams = c_vec.fit_transform(corpus)
    count_values = ngrams.toarray().sum(axis=0)
    vocab = c_vec.vocabulary_
    df_ngram = pandas.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'bigram/trigram'})
    
    return df_ngram


# coletar os comentários

In [3]:
database = getComments("./General_Data.csv","2021-09-01","2021-10-01","3.0.0")

# buscar por comentários similares

In [19]:
similarities_list = searchSimilar('nao consigo recuperar minha senha', database)
d = similarities_list.sort_values('similaridade', ascending=False)


nao consigo recuperar minha senha


In [20]:
d

,Unnamed: 0,date,content,score,version,thumbs,similaridade,comentarios_limpos
483,154,2021-09-22 08:16:04,Não consigo recuperar a senha,3,3.0.0,0,[[1.0000000000000002]],senha
397,65,2021-09-28 10:27:24,Não estou conseguindo recuperar Minha senha O ...,3,3.0.0,0,[[0.8995683899185091]],senha
216,69,2021-09-25 13:16:19,Não consigo minha senha,2,3.0.0,0,[[0.7082079097355087]],senha
86,117,2021-10-01 09:48:30,"O aplicativo é péssimo, não estou conseguindo ...",1,3.0.0,0,[[0.609924518599725]],aplicativo senha
626,101,2021-09-28 10:24:57,deu certo recuperar minha senha muito bom,4,3.0.0,0,[[0.5834141223966954]],certar senha
...,...,...,...,...,...,...,...,...
426,96,2021-09-25 22:46:59,Bom,3,3.0.0,0,[[0.0]],Bom
427,97,2021-09-25 20:09:08,Desisti do cadastro. Não confio em compartilha...,3,3.0.0,0,[[0.0]],Desisti cadastrar dar Internet banco
428,98,2021-09-25 18:04:40,não funciona o reconhecimento facial,3,3.0.0,0,[[0.0]],reconhecimento
429,99,2021-09-25 17:47:27,bom,3,3.0.0,0,[[0.0]],


483                        Não consigo recuperar a senha
397    Não estou conseguindo recuperar Minha senha O ...
216                              Não consigo minha senha
86     O aplicativo é péssimo, não estou conseguindo ...
626            deu certo recuperar minha senha muito bom
                             ...                        
426                                                  Bom
427    Desisti do cadastro. Não confio em compartilha...
428                 não funciona o reconhecimento facial
429                                                  bom
914                                            muito bom
Name: content, Length: 755, dtype: object


In [7]:
d = limparComentarios(database)
print(d['comentarios_limpos'])

29     Péssimo horror cadastrar servidor câmera recon...
30                      reconhecimento contar CIV rápido
31     Horrível Acesso gov.br navegador Chrome usuári...
32     contar reconhecimento mensagem errar servidor ...
33     aplicativo senha acesso dificuldade senha)poré...
                             ...                        
910                                                     
911                                                     
912                                                     
913                                                     
914                                                     
Name: comentarios_limpos, Length: 755, dtype: object


In [ ]:
n = extrairNgrams(d, 1, 1)

In [ ]:
print(n)